# Using Hugging Face Transformer

The transformers library was designed to create a single api that will allow a user to load, train and save ML models.

The previous section covered the pipeline object which allowed for fast, simple inference on a wide variety of tasks using models available on the HF platform.

We can dive deeper into this though. For this lesson we are going to perform an end-to-end example. We will use a model and a tokenzier together to replicate the pipeline() function. Next we will dive into the model and its configuration classes - showing you how to load a model and how it processes numerical inputs to output predictions

Then we will examine the tokenizer API. Tokenizers take care of the first and last processing steps, handling the converison from text o numerical inputs and the conversion back to text.

For this example we will construct a pipeline that will take an input text and return summerizations of the text.

In [1]:
import json

# Now let's grab some of our data.
# For this we will be using the "press_release_pdf" key for each element in data
# In the instance of this dataset the value is a string containing the press release from an art show
# Look, i know the key string is not the ebst, i'll fix it.....


json_file = "tzvet_data.json"

with open(json_file, "r") as file:
    data = json.load(file)

words = data[0]["press_release_pdf"]

raw_inputs = []

if len(words) >= 4:
    part1 = words[:len(words) // 4]
    part2 = words[len(words) // 4:2 * len(words) // 4]
    part3 = words[2 * len(words) // 4:3 * len(words) // 4]
    part4 = words[3 * len(words) // 4:]

    raw_inputs.extend([part1, part2, part3, part4])

print(len(raw_inputs))


4


#### AutoTokenizer
 Next we need to figure out what type of problem we are tring to solve. In our case we want to generate labels for images that are contained in an art show this can be done using two methods - summarization or text generation. We will examine both, but, my gut is leaning toward text generation. We want poetry rather thana computer trying to rationalize about someones poetry.

Let's roll!

In [2]:
from transformers import AutoTokenizer

# First we will try using a mistral 7B LLM - mistralai/Mistral-7B-v0.1 
checkpoint = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer, tokenizer("This is a test")["input_ids"], tokenizer.encode("This is a test")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


(LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.1', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'additional_special_tokens': ['<unk>', '<s>', '</s>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
 	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 },
 [1, 851, 349, 264, 1369],
 [1, 851, 349, 264, 1369])

--------------------------
Looks like the AutoTokenizer piicked out the "LlamaTokenizerFast" - let's take a look at some of the documentation for it

[LLamaTokenizerFast Source Code](https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/tokenization_llama_fast.py)

--------------------------

Once we have a tokenizer we can directly pass our sentences to it and we'll get back a dictionary that is ready to feed to our model!

Now all we have to do is convert the list of input IDs to tensors. What is crucial though is to know what framework our model is built on. Often this can be found on the models HF page.

In our case, we are using the [mistralai 7B model](https://huggingface.co/mistralai/Mistral-7B-v0.1?text=Dead+skin+subject+to+gravity%3B+fossil+armament+of+a+soldier+as+a+remnant+of+war+conflict%3B+abandoned+bee+combs+without+their+companies+and+infected+by+a+pest%3B+%27condensation%27+of+automobile+waste+artificially+created+stalactites%3B+milk+in+a+bowl+on+a+stove%3B+the+Metropolis+of+London%27s+asphalted+streets+turned+into+a+city+of+blinds%3B+the+sun%27s+rays+on+a+windowpane%3B+a+power+cable+lying+on+the+ground%3B+the+exposed+skeleton+of+a+chicken%3B+a+glass+of+water+on+a+table%3B+a+snake%27s+skin%3B+a+single+button+taken+from+a+jacket%3B+a+screen+showing+a+still+image.+All+these+images+are+those+of+the+%27real+world%27%2C+but+they+are+not+%27naturally%27+real.+They+have+been+altered+by+the+artist%27s+hands+and+%27mutated%27+as+much+as+possible.+The+result+of+this+%27mutation%27+is+to+highlight+the+aesthetic+value+of+the+real+world%2C+and+to+confirm+its+fragility%2C+as+much+as+to+prove) -  which states that it is a PyTorch model.

When we then call the tokenizer on our raw_inputs we must specify what framework we are working with. For us we will assign it to 'pt' - PyTorch


In [3]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>']

In [4]:
# This tokenizer did not have a padding token. Hmm, I'm a little worried this may have led me astray - we'll see
pad_token = '<pad>'
tokenizer.add_special_tokens({'pad_token': pad_token}) 

# Generate our tokenized inputs and attention mask! Notice the padding is added to the left.
# See LlamaTokenizerFast source code to know more - follow the rabbit
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors='pt')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


#### AutoModel

Now let's load in the model itself. We'll be using AutoModel to help us identify the appropriate model for our checkpoint

In [5]:
from transformers import AutoModel

# It is crucial that the version of transformers we are using is 4.34.0 or greater in order to use mistral 7B!!!

model = AutoModel.from_pretrained(checkpoint)
model

: 

In [ ]:
import transformers
transformers.__version__

'4.32.0'